In [35]:
import json
import requests
import pandas as pd
import datetime as dt

from tqdm.notebook import tqdm

In [36]:
data_hoje = dt.date.today()
data_hoje = str(data_hoje)
print(data_hoje)

2021-05-23


In [37]:
resposta = requests.get('https://api.helium.io/v1/hotspots')
print(str(resposta))
dados = resposta.json()
cursor = dados['cursor']
hotspots = dados['data']


<Response [200]>


In [38]:
while str(resposta) == '<Response [200]>':
    resposta = requests.get('https://api.helium.io/v1/hotspots?cursor=' + cursor)
    print(resposta)
    dados = resposta.json()
    hotspots += dados['data']
    try: cursor = dados['cursor']
    except KeyError: break
    print(len(hotspots))


<Response [200]>
2000
<Response [200]>
3000
<Response [200]>
4000
<Response [200]>
5000
<Response [200]>
6000
<Response [200]>
7000
<Response [200]>
8000
<Response [200]>
9000
<Response [200]>
10000
<Response [200]>
11000
<Response [200]>
12000
<Response [200]>
13000
<Response [200]>
14000
<Response [200]>
15000
<Response [200]>
16000
<Response [200]>
17000
<Response [200]>
18000
<Response [200]>
19000
<Response [200]>
20000
<Response [200]>
21000
<Response [200]>
22000
<Response [200]>
23000
<Response [200]>
24000
<Response [200]>
25000
<Response [200]>
26000
<Response [200]>
27000
<Response [200]>
28000
<Response [200]>
29000
<Response [200]>
30000
<Response [200]>
31000
<Response [200]>
32000
<Response [200]>
33000
<Response [200]>
34000
<Response [200]>
35000
<Response [200]>
36000
<Response [200]>
37000
<Response [200]>
38000
<Response [200]>
39000
<Response [200]>
40000
<Response [200]>
41000
<Response [200]>


In [ ]:
# criação da planilha do Excel com os dados brutos dos hotspots
hotspots_dataframe = pd.DataFrame(hotspots)
writer = pd.ExcelWriter('hotspots_bruto.xlsx')
hotspots_dataframe.to_excel(writer)
writer.save()
hotspots_dataframe


In [42]:
# filtragem dos hotspots "inativos"
bloco_atual = requests.get('https://api.helium.io/v1/blocks/height').json()['data']['height']
print(bloco_atual)


856996


In [57]:
tolerancia = 32
hotspots_relevantes = list()
for hotspot in hotspots:
    altura = hotspot['status']['height']
    if altura != None:
        if altura >= bloco_atual - tolerancia: hotspots_relevantes.append(hotspot)

print(len(hotspots_relevantes))


7556


In [ ]:
hotspots_dados = list()
max_time = "%sT00:00:00Z" % data_hoje

indice = 0
for hotspot in tqdm(hotspots_relevantes):
    
    dados = dict()
    dados['nome'] = hotspot['name']
    dados['data'] = hotspot['timestamp_added']
    dados['endereco'] = hotspot['address']
    dados['altura'] = hotspot['status']['height']
    
    endereco = dados['endereco']
    params = {'max_time': max_time, 'min_time': "2018-08-27T00:00:00Z"}
    dados_retorno = requests.get('https://api.helium.io/v1/hotspots/:%s/rewards/sum' % endereco, params)
    dados_retorno = dados_retorno.json()['data']
    
    dados.update(dados_retorno)
    
    hotspots_dados.append(dados)
    
    if indice % 50 == 0:
        with open('hotspots_dados.json', 'w') as file:
            json.dump({'dados': hotspots_dados}, file)
    
    indice += 1
    
hotspots_dataframe = pd.DataFrame(hotspots_dados)
writer = pd.ExcelWriter('hotspots_dados.xlsx')
hotspots_dataframe.to_excel(writer)
writer.save()
hotspots_dados
    